In [1]:
import numpy as np
import pandas as pd
import string
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
import matplotlib
import langid
import re

from matplotlib import pyplot as plt
# import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
dataFull = pd.read_csv("../Instagram_Like_Predictor/InstagramDatasetFinalVersion.csv",index_col = 0)
dataFull.head()
dataText = dataFull.loc[:,['hashtags','mentions','location','edge_media_to_caption']]
dataLabels = dataFull.loc[:,['mean_likes']]

dataText['mentions'].replace('[]','',inplace=True)
dataText['hashtags'].replace('[]', '', inplace=True)
#fill nan w str
dataText.fillna({'location':''},inplace=True)
dataText.fillna({'edge_media_to_caption':''},inplace = True)
# convert to str
dataText['edge_media_to_caption']= dataText['edge_media_to_caption'].astype(str)
dataText['location'] = dataText['location'].astype(str)
dataText['mentions'] = dataText['mentions'].astype(str)
dataText['hashtags'] = dataText['hashtags'].astype(str)

dataText.head()
dataText.to_csv('NLP.csv', index=None, header= True)
dataText.info()



<class 'pandas.core.frame.DataFrame'>

📧takashi.yasui1980@gmail.com
Data columns (total 4 columns):
hashtags                 20668 non-null object
mentions                 20668 non-null object
location                 20668 non-null object
edge_media_to_caption    20668 non-null object
dtypes: object(4)
memory usage: 807.3+ KB


In [3]:
# bag of words, tfidf, autoencoder, 
def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

dataText['edge_media_to_caption'] = dataText['edge_media_to_caption'].apply(remove_punctuation)
dataText['location'] = dataText['location'].apply(remove_punctuation)
dataText['mentions'] = dataText['mentions'].apply(remove_punctuation)
dataText['hashtags'] = dataText['hashtags'].apply(remove_punctuation)
dataText.head()
#dataText.to_csv('NLP.csv',index = None, header=True)


,hashtags,mentions,location,edge_media_to_caption
biography,,,,
NaN,,,,OCLOCK이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요❤️
NaN,,,,즐거운 화이트데이메이트데이🧛‍♂️❤️
NaN,,,,OCLOCK 곧 12시죠 두구두구 티저 이미지 공개 😆
NaN,,,,None
NaN,,,,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요


In [4]:
# Remove Emojis id lan
dataText["edge_media_to_caption"] = dataText['edge_media_to_caption'].str.replace('[^\w\s]','') #remove emojis

def identify_language(row):
    lang = langid.classify(row['edge_media_to_caption'])
    return lang[0]
#depends on if langid wants to work
dataText['language'] = dataText.apply(identify_language,axis=1)
dataText.head(20)

,hashtags,mentions,location,edge_media_to_caption,language
biography,,,,,
NaN,,,,OCLOCK이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko
NaN,,,,즐거운 화이트데이메이트데이,ko
NaN,,,,OCLOCK 곧 12시죠 두구두구 티저 이미지 공개,ko
NaN,,,,None,en
NaN,,,,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko
NaN,,,,빼꼼 메이 사랑해요,ko
NaN,,,,복 많이받으세요,ko
NaN,,,,이젠 어렵지 않아요,ko
NaN,,,,다음번에는 예고없이 바로 방송킬꺼에요,ko


In [5]:
# bag of words, tfidf, autoencoder, 
#everything lower case and removed stopwords
from nltk.corpus import stopwords
sw = stopwords.words('english')
np.array(sw)
def stopwords(text):
    text = [word.lower()for word in text.split() if word.lower() not in sw]
    return " ".join(text)
dataText['edge_media_to_caption'] = dataText['edge_media_to_caption'].apply(stopwords)

def lower(text):
    text = [word.lower() for word in text.split()]
    return " ".join(text)

dataText['location']= dataText['location'].apply(lower)
dataText['hashtags']= dataText['hashtags'].apply(lower)
dataText['mentions'] = dataText['mentions'].apply(lower)
dataText.head()
dataText.to_csv('NLP.csv',index = None, header=True)




In [6]:
#Translate then stem words then apply vectorizer

from googletrans import Translator
translator = Translator()
def translate(row):
    translator = Translator()
    if str(row['language']) =='en':
        return str(row['edge_media_to_caption'])
    else:
        try:
            english = translator.translate(str(row['edge_media_to_caption']),src = row['language'],dest = 'en').text
            return english
        except:
            return ''
dataText['english'] = dataText.apply(translate,axis=1)

dataText.head()

,hashtags,mentions,location,edge_media_to_caption,language,english
biography,,,,,,
NaN,,,,oclock이 발매되었습니다 여러분들의 많은 관심과 사랑부탁드릴께요,ko,oclock has been released.
NaN,,,,즐거운 화이트데이메이트데이,ko,Happy White Day Mate Day
NaN,,,,oclock 곧 12시죠 두구두구 티저 이미지 공개,ko,oclock shortly 12 o'clock
NaN,,,,none,en,none
NaN,,,,오늘 발렌타인데이인데 초콜렛말고 제 하트받으실래요,ko,Today is Valentine's Day.


In [7]:
dataText.to_csv('NLP.csv',index = None, header=True)